In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from PIL import Image

In [ ]:
import pandas as pd
data = pd.read_pickle("data_location_df.pickle")
import numpy as np

In [ ]:
dt = pd.read_csv("uscities.csv")
state_US=dt.state_name.unique()

In [ ]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1e5

In [ ]:
state_list=data.state.tolist()
state_int=[]
for i in range (len(state_list)):
    if(state_list[i] in state_US):
      state_int.append(state_list[i])

In [ ]:
states = set(state_int)

In [ ]:
df = data.drop(data[~(data.state.isin(states))].index)
s=df.state.unique()
print(len(s))
index_list = list(s)
state_list=df.state.tolist()
state_index=[]
for i in range (len(state_list)):
  state_index.append(index_list.index(state_list[i]))
df["state_index"] = state_index

print(df)

50
                 user           tweet_time                coordinate  \
0       USER_79321756  2010-03-03T04:15:26  (47.528139, -122.197916)   
1       USER_79321756  2010-03-03T04:55:32  (47.528139, -122.197916)   
2       USER_79321756  2010-03-03T05:13:34  (47.528139, -122.197916)   
3       USER_79321756  2010-03-03T05:28:02  (47.528139, -122.197916)   
4       USER_79321756  2010-03-03T05:56:13  (47.528139, -122.197916)   
...               ...                  ...                       ...   
377611  USER_8c704efa  2010-03-06T17:51:27   (39.955937, -75.159995)   
377612  USER_8c704efa  2010-03-06T18:49:45   (39.955937, -75.159995)   
377613  USER_8c704efa  2010-03-06T19:03:48   (39.955937, -75.159995)   
377614  USER_8c704efa  2010-03-06T19:19:05   (39.955937, -75.159995)   
377615  USER_8c704efa  2010-03-07T05:50:25    (40.67134, -73.855996)   

              lat         long  \
0       47.528139  -122.197916   
1       47.528139  -122.197916   
2       47.528139  -122.197916

In [ ]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
X, Y = df.processed_tweet.tolist(), df.state_index.tolist()
cv = CountVectorizer(max_df=0.95, min_df=2,
                                     max_features=10000,
                                     stop_words='english')
X_vec = cv.fit_transform(X)

res = dict(zip(cv.get_feature_names(),
               mutual_info_classif(X_vec, Y, discrete_features=True)
               ))
print(res)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{'00': 0.0007732286220016372, '000': 0.0001376196646720265, '01': 6.519389370007758e-05, '02': 6.21293568683253e-05, '03': 7.723136592100284e-05, '04': 9.45365648935095e-05, '05': 8.11240671395283e-05, '06': 0.0001058764547474274, '07': 8.689981212098654e-05, '08': 3.637392706858385e-05, '09': 7.818395089970282e-05, '0_0': 5.863641493999817e-05, '0_o': 7.949535794965598e-05, '0n': 6.608218286318261e-05, '0pen': 5.4656158224170406e-05, '10': 0.00014459817668126422, '100': 0.00013087277498922005, '1000': 6.251193200895014e-05, '101': 3.368996444701043e-05, '103': 3.5906233554685484e-05, '1030': 2.7639703379621255e-05, '105': 8.56562516169398e-05, '106': 7.191021267509827e-05, '10am': 3.7304388532479546e-05, '10pm': 4.580940246919358e-05, '10th': 7.312088671654774e-05, '10x': 2.48299275362931e-05, '11': 0.00011082130499056263, '110': 5.9795352636606804e-05, '111': 4.568644250056813e-05, '112': 4.479418127724807e-05, '1130': 3.6566443662121515e-05, '11am': 4.6975616111990795e-05, '11pm': 5

In [ ]:
ent_des=dict(sorted(res.items(), key=lambda item: item[1],reverse=True))
MI_new = pd.DataFrame(list(ent_des.items()), columns = ['Attribute','MI'])

MI = MI_new.drop(MI_new[MI_new.MI <0.0001 ].index)
print(MI)

         Attribute        MI
0              lls  0.004743
1     inhighschool  0.002654
2             haha  0.002652
3              lml  0.002440
4            lmaoo  0.002348
...            ...       ...
2255         tried  0.000100
2256        genius  0.000100
2257          star  0.000100
2258       kitchen  0.000100
2259            38  0.000100

[2260 rows x 2 columns]


In [ ]:
BOW=MI.Attribute.tolist()
BoW_set = set(BOW)

In [ ]:
vec_list=[]
for i in range(X_vec.shape[0]):
    dense=X_vec[i].todense()
    b=np.where(dense!=0)
    vec=[0]*len(BOW)
    for k in b[1]:
       term= cv.get_feature_names()[k]
       if(term in BoW_set) :
        vec[BOW.index(term)]=1
    vec_list.append(vec)
    print(i)

In [ ]:
X, Y = data.processed_tweet.tolist(), data.state_index.tolist()
cv = CountVectorizer(max_df=0.95, min_df=2,
                                     max_features=10000,
                                     stop_words='english')
X_vec = cv.fit_transform(X)

res = dict(zip(cv.get_feature_names(),
               mutual_info_classif(X_vec, Y, discrete_features=True)
               ))
print(res)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{'00': 0.0008293465058570429, '000': 0.00014605830894463844, '01': 9.749626240898518e-05, '02': 9.894591280654066e-05, '03': 0.00035559363947873523, '04': 0.0001942458796991796, '05': 0.00010191902504571909, '06': 0.00011276668832264809, '07': 9.596045618027241e-05, '08': 5.8683352178286635e-05, '09': 8.099200452713102e-05, '0_0': 6.24543717764638e-05, '0_o': 7.962171987026702e-05, '0n': 6.76584850641668e-05, '10': 0.00025234693019146094, '100': 0.00014128515581987388, '1000': 6.34641943012357e-05, '101': 4.275965645368277e-05, '103': 3.784468285003795e-05, '1030': 2.923646350471119e-05, '105': 8.545761548236068e-05, '106': 7.26141255723812e-05, '10am': 5.1971941067307415e-05, '10pm': 4.920377744609257e-05, '10th': 7.655059242127797e-05, '11': 0.00016687513722328756, '111': 4.524623361271662e-05, '112': 4.835165538084459e-05, '1130': 3.633492611917673e-05, '11am': 4.820421045507601e-05, '11pm': 5.126185479662137e-05, '11th': 5.515249857337096e-05, '12': 0.000158057724938467, '120': 5.9

In [ ]:
dt = pd.read_csv("uscities.csv")

In [ ]:
state_US=dt.state_name.unique()
print(state_US)

['New York' 'California' 'Illinois' 'Florida' 'Texas' 'Pennsylvania'
 'Georgia' 'District of Columbia' 'Massachusetts' 'Arizona' 'Michigan'
 'Washington' 'Minnesota' 'Colorado' 'Maryland' 'Nevada' 'Missouri'
 'Oregon' 'Puerto Rico' 'Ohio' 'Indiana' 'North Carolina' 'Virginia'
 'Wisconsin' 'Rhode Island' 'Utah' 'Tennessee' 'Kentucky' 'Louisiana'
 'Connecticut' 'Oklahoma' 'Hawaii' 'Nebraska' 'New Mexico' 'Alabama'
 'South Carolina' 'Iowa' 'Kansas' 'Arkansas' 'Idaho' 'Mississippi'
 'New Jersey' 'Alaska' 'New Hampshire' 'Maine' 'North Dakota'
 'West Virginia' 'South Dakota' 'Delaware' 'Montana' 'Vermont' 'Wyoming']


In [ ]:
state_int=[]
for i in range (len(state_list)):
    if(state_list[i] in state_US):
      state_int.append(state_list[i])
print(state_int)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

